# Life Cycle Assessment of the CeRAS aircraft

The aim of this tutorial is to evaluate the environmental impacts of the CeRAS aircraft using the LCA module of FAST-OAD. The LCA module is applied here as a post-processing step only, after sizing the CeRAS aircraft. However, this module can be used like any other FAST-OAD's plugin to run any MDAO problem you want, for example to optimise the design with environmental criteria.

More information on the CeRAS case study can be found [here](https://github.com/fast-aircraft-design/FAST-OAD_CS25/blob/main/src/fastoad_cs25/notebooks/02_CeRAS_case_study/CeRAS_case_study.ipynb).

This notebook requires the [CS-25 package of FAST-OAD](https://github.com/fast-aircraft-design/FAST-OAD_CS25), which can be installed using `pip install --upgrade fast-oad-cs25`, as well as a valid license from ecoinvent (refer to installation procedure in the README of this plugin).

## Imports

In [ ]:
import os.path as pth
import sys
import logging
import fastoad.api as oad
sys.path.append(pth.abspath("."))

## Working directories and reference files

In [ ]:
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
OAD_CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "oad_sizing.yml")
OAD_SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "CeRAS_reference_data.xml")
LCA_CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "operational_missions_lca.yml")

## OAD Process

In a first step, the CeRAS aircraft is sized using FAST-OAD, following the approach provided in the [CeRAS case study notebook](https://github.com/fast-aircraft-design/FAST-OAD_CS25/blob/main/src/fastoad_cs25/notebooks/02_CeRAS_case_study/CeRAS_case_study.ipynb).

In [ ]:
# Generate input file for OAD problem
oad_input_file = oad.generate_inputs(OAD_CONFIGURATION_FILE, OAD_SOURCE_FILE, overwrite=True)

In [ ]:
# Display input file
oad.variable_viewer(oad_input_file)

In [ ]:
# Run problem
MTOW_problem = oad.evaluate_problem(OAD_CONFIGURATION_FILE, overwrite=True)

In [ ]:
# Display outputs
OAD_OUTPUT_FILE = MTOW_problem.output_file_path
oad.variable_viewer(OAD_OUTPUT_FILE)

## FAST-OAD configuration file and LCA setup

The LCA module can be used like any other FAST-OAD plugin using the id `fastoad.plugin.lca` in the [configuration file](./data/operational_missions_lca.yml):

```yaml
environmental_impacts:
        id: fastoad.plugin.lca
        configuration_file: './CeRAS_LCA.yml'  # path to the LCA configuration file
        axis: 'phase'  # return impacts per contributor (e.g. "phase"), according to tags in the LCA model
        normalization: False  # normalize results
        weighting: False  # weight and aggregate results into a single score
        nonfloat_params:  # provide values for non-float LCA parameters
            model: 'remind'
            pathway: 'SSP2-PkBudg1150'
```

The module comes with a number of options, of which the path to the [LCA definition file](./data/CeRAS_LCA.yml) (`configuration_file`) is mandatory.
This file enables to define the LCA problem, including:
1. The version & system model of [ecoinvent](https://support.ecoinvent.org/releases-overview) to install and use in the LCA
2. [Optional] For future-oriented studies, the prospective scenarios following the [premise](https://premise.readthedocs.io/en/latest/extract.html) nomenclature (IAM model, pathway, and year).
3. The LCA model itself, using a tree-like structure.
4. [Optional] Parameters metadata, e.g. to set units for LCA parameters to ensure consistency with FAST-OAD's variables
5. The LCIA methods

More information and tutorials on how to set up an LCA problem are available in the dedicated Python package [LCA-Modeller](https://github.com/AeroMAPS/lca-modeller) developed in support of this project.

## Generate input files and set values

Once the configuration file and the LCA file are ready, you can generate the input file as with any FAST-OAD problem. **The first call will take some time, depending on the size of your LCA model**. This is because the LCA model and the parametric LCIA functions are being generated. However, this is only done once and, unless you change the LCA configuration file, subsequent runs will be much faster. For more information on parametric LCA in Python, see [lca-algebraic](https://lca-algebraic.readthedocs.io/).

In [ ]:
# Generate input file for LCA

# From OAD process run in this notebook...
#lca_input_file = oad.generate_inputs(LCA_CONFIGURATION_FILE, OAD_OUTPUT_FILE, overwrite=True)

# ..Or from existing file from previous OAD run
OAD_OUTPUT_FILE = pth.join(DATA_FOLDER_PATH, 'operational_missions_lca_in.xml')

lca_input_file = oad.generate_inputs(LCA_CONFIGURATION_FILE, OAD_OUTPUT_FILE, overwrite=True)

You can now visualise the input file and fill in the missing values needed to evaluate the problem. Don't forget to save the file before going any further.

In [ ]:
oad.variable_viewer(lca_input_file)

## Running the LCA

Let's run the LCA!

In [ ]:
LCA_problem = oad.evaluate_problem(LCA_CONFIGURATION_FILE, overwrite=True)

## Results Analysis

You can now visualise the results and plot any nice LCA figure you want. Some examples are provided in notebook [02_Plots_For_Transportation_Part_D.ipynb](02_Plots_For_Transportation_Part_D.ipynb)

In [ ]:
LCA_OUTPUT_FILE = LCA_problem.output_file_path
oad.variable_viewer(LCA_OUTPUT_FILE)

## Bonus: visualisation of the LCA model

FAST-OAD-LCA stores the parametric LCA model under the name `model` in the database `Foreground DB`. Once retrieved, it is possible to visualise the processes and exchanges involved in your model in the form of a process tree.

In [ ]:
import lca_algebraic as agb
from lca_modeller.gui.plots import process_tree

my_lca_model = agb.findActivity('model', db_name='Foreground DB')
process_tree(my_lca_model, foreground_only=True)

Alternatively, you can visualise it in the form of a table:

In [ ]:
from lca_modeller.helpers import list_processes
list_processes(my_lca_model)

Finally, you can display all the parameters involved in your LCA model. Three of these parameters are specific to prospective scenarios and generated automatically: `model`, `pathway`, and `year`. The first two must be set as options in the FAST-OAD configuration file, while the year can be entered directly in the input files.

In [ ]:
agb.list_parameters()